### Objective: Explore the relationship between trader performance and market sentiment; surface actionable insights for trading strategies.

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [4]:
OUTPUT_DIR = "assignment folder"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [6]:
def safe_read_csv(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path}")
    return pd.read_csv(path)

def normalize_cols(df):
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df.columns = [c.replace(" ", "_").replace("-", "_") for c in df.columns]
    df.columns = [c.lower() for c in df.columns]
    return df

def find_datetime_col(df, candidates):
    for name in candidates:
        if name in df.columns:
            # try parse sample
            try:
                tmp = pd.to_datetime(df[name], errors='coerce')
                if tmp.notna().any():
                    return name
            except Exception:
                continue
    return None

def ensure_utc(dt_series):
    # converting to datetime and force UTC tz-aware where possible
    s = pd.to_datetime(dt_series, errors='coerce')
    # if already tz-aware, converting to UTC
    if s.dt.tz is not None:
        s = s.dt.tz_convert("UTC")
    else:
        # localizing naive to UTC 
        s = s.dt.tz_localize("UTC")
    return s


####  1) Loading

In [10]:
trades=pd.read_csv(r"C:\Users\saram\Downloads\historical_data.csv")
sent=pd.read_csv(r"C:\Users\saram\Downloads\fear_greed_index.csv")

In [12]:
trades.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


In [14]:
sent.head()

,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


#### 2) Normalize column names

In [12]:
def normalize_cols(df):
    """Normalizes column names by:
    - stripping extra spaces
    - converting to lowercase
    - replacing spaces with underscores"""
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(' ', '_')
    )
    return df


In [14]:
trades = normalize_cols(trades)
sent = normalize_cols(sent)


#### 3) Data quality checks

In [16]:
def dqa_print(df, name):
    print(f"--- {name} DQ ---")
    print("rows:", len(df))
    print("columns:", len(df.columns))
    print("duplicates:", int(df.duplicated().sum()))
    print("missing per column (top 10):")
    print(df.isna().sum().sort_values(ascending=False).head(10))
    print()

dqa_print(trades, "Trades")
dqa_print(sent, "Sentiment")


pd.DataFrame({"column":list(trades.columns), "missing":trades.isna().sum().values, "dtype":trades.dtypes.astype(str).values}).to_csv(
    os.path.join(OUTPUT_DIR,"trades_columns_report.csv"), index=False)
pd.DataFrame({"column":list(sent.columns), "missing":sent.isna().sum().values, "dtype":sent.dtypes.astype(str).values}).to_csv(
    os.path.join(OUTPUT_DIR,"sent_columns_report.csv"), index=False)


--- Trades DQ ---
rows: 211224
columns: 16
duplicates: 0
missing per column (top 10):
account            0
coin               0
execution_price    0
size_tokens        0
size_usd           0
side               0
timestamp_ist      0
start_position     0
direction          0
closed_pnl         0
dtype: int64

--- Sentiment DQ ---
rows: 2644
columns: 4
duplicates: 0
missing per column (top 10):
timestamp         0
value             0
classification    0
date              0
dtype: int64



#### 4) Detect and convert timestamps to UTC tz-aware

In [18]:
def find_datetime_col(df, candidates):
    """Finds the first column in df whose name matches one in candidates.
    Matching is case-insensitive."""
    for col in df.columns:
        if col.lower() in [c.lower() for c in candidates]:
            return col
    return None


In [20]:
def ensure_utc(series):
    """Converts a pandas Series to timezone-aware UTC datetime."""
    # Convert to datetime if not already
    series = pd.to_datetime(series, errors='coerce', utc=True)

    # If already timezone-naive, localize to UTC
    if series.dt.tz is None:
        series = series.dt.tz_localize('UTC')
    else:
        series = series.dt.tz_convert('UTC')

    return series


In [43]:
trade_time_candidates = ["time","timestamp","date","datetime","trade_time"]
sent_time_candidates = ["date","datetime","timestamp","time"]

trade_time_col = find_datetime_col(trades, trade_time_candidates)
sent_time_col = find_datetime_col(sent, sent_time_candidates)

if trade_time_col is None or sent_time_col is None:
    raise RuntimeError(f"Could not detect time columns. Detected trade_time_col={trade_time_col}, sent_time_col={sent_time_col}")

print("Detected time columns:", trade_time_col, sent_time_col)

# convert to tz-aware UTC
trades[trade_time_col] = ensure_utc(trades[trade_time_col])
sent[sent_time_col] = ensure_utc(sent[sent_time_col])

# drop rows missing times
trades = trades[~trades[trade_time_col].isna()].copy()
sent = sent[~sent[sent_time_col].isna()].copy()

Detected time columns: timestamp timestamp


#### 5) Normalize numeric columns and identify key fields

In [26]:
# heuristics for column names
def find_first(cols, patterns):
    for p in patterns:
        if p in cols:
            return p
    return None

cols = trades.columns.tolist()
# detect PnL-like column
pnl_col = next((c for c in cols if "pnl" in c.lower() or "profit" in c.lower()), None)
size_col = next((c for c in cols if any(x==c for x in ["size","qty","quantity","volume","amount"]) or "size" in c.lower()), None)
price_col = next((c for c in cols if any(x in c.lower() for x in ["price","execution_price","exec_price","entry_price","exit_price"])), None)
side_col = next((c for c in cols if c.lower() in ["side","direction","buy_sell","buy/sell"] or "side" in c.lower()), None)
account_col = next((c for c in cols if any(x in c.lower() for x in ["account","acct","client","trader","user"])), None)
symbol_col = next((c for c in cols if any(x in c.lower() for x in ["symbol","coin","pair","instrument"])), None)
leverage_col = next((c for c in cols if "leverage" in c.lower() or "lev"==c.lower()), None)

print("Detected: pnl=", pnl_col, "size=", size_col, "price=", price_col, "side=", side_col, "account=", account_col, "symbol=", symbol_col, "leverage=", leverage_col)
# Coerce to numeric engineered columns
trades["_pnl"] = pd.to_numeric(trades[pnl_col], errors='coerce') if pnl_col else np.nan
trades["_size"] = pd.to_numeric(trades[size_col], errors='coerce') if size_col else np.nan
trades["_price"] = pd.to_numeric(trades[price_col], errors='coerce') if price_col else np.nan
trades["_leverage"] = pd.to_numeric(trades[leverage_col], errors='coerce') if leverage_col else np.nan

# side normalization
if side_col:
    trades["_side"] = trades[side_col].astype(str).str.lower().str.strip()
else:
    trades["_side"] = np.nan

# notional and return
trades["_notional"] = trades["_size"] * trades["_price"]
trades["_return_pct"] = trades["_pnl"] / trades["_notional"]
trades["_return_pct"].replace([np.inf, -np.inf], np.nan, inplace=True)
trades["_profitable"] = trades["_pnl"] > 0

# holding time if open & close timestamps exist
open_candidates = [c for c in cols if "open_time" in c or "open"==c or "start_time" in c]
close_candidates = [c for c in cols if "close_time" in c or "close"==c or "end_time" in c]
open_time_col = open_candidates[0] if open_candidates else None
close_time_col = close_candidates[0] if close_candidates else None
if open_time_col and close_time_col and open_time_col in trades.columns and close_time_col in trades.columns:
    trades[open_time_col] = ensure_utc(trades[open_time_col])
    trades[close_time_col] = ensure_utc(trades[close_time_col])
    trades["_holding_time_sec"] = (trades[close_time_col] - trades[open_time_col]).dt.total_seconds()
else:
    trades["_holding_time_sec"] = np.nan

# saving a small sample of engineered trades
trades[[trade_time_col,"_pnl","_size","_price","_notional","_return_pct","_profitable","_leverage","_side"]].head(20).to_csv(
    os.path.join(OUTPUT_DIR,"trades_engineered_sample.csv"), index=False)

Detected: pnl= closed_pnl size= size_tokens price= execution_price side= side account= account symbol= coin leverage= _leverage


C:\Users\saram\AppData\Local\Temp\ipykernel_10372\3184332210.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  trades["_return_pct"].replace([np.inf, -np.inf], np.nan, inplace=True)


In [24]:
# Detect trade timestamp column
possible_trade_time_cols = ["timestamp", "time", "trade_time", "datetime", "date"]
trade_time_col = next((col for col in possible_trade_time_cols if col in trades.columns), None)

if trade_time_col is None:
    raise ValueError(f"No trade time column found in trades. Columns found: {trades.columns.tolist()}")

trades[trade_time_col] = ensure_utc(trades[trade_time_col])

# saving a small sample of engineered trades
trades[[trade_time_col, "_pnl", "_size", "_price", "_notional", "_return_pct", "_profitable", "_leverage", "_side"]] \
    .head(20) \
    .to_csv(os.path.join(OUTPUT_DIR, "trades_engineered_sample.csv"), index=False)


#### 6) Prepare sentiment numeric & classification

In [28]:
possible_sent_val_cols = [c for c in sent.columns if any(k in c for k in ["value","index","score","val","fear","greed"])]
sent_val_col = None
for c in possible_sent_val_cols:
    sent[c] = pd.to_numeric(sent[c], errors='coerce')
    if sent[c].notna().any():
        sent_val_col = c
        break
if sent_val_col is None:
    # fallback
    for c in sent.columns:
        if pd.api.types.is_numeric_dtype(sent[c]):
            sent_val_col = c
            break
if sent_val_col is None:
    raise RuntimeError("No numeric sentiment value column found in sentiment dataset.")
print("Using sentiment value column:", sent_val_col)

possible_class_cols = [c for c in sent.columns if any(k in c for k in ["class","classification","label"])]
sent_class_col = possible_class_cols[0] if possible_class_cols else None

Using sentiment value column: value


#### 7) Align sentiment with trades

In [30]:
possible_class_cols = ["classification", "category", "sentiment_class", "label"]
sent_class_col = None
for col in sent.columns:
    if col.lower() in possible_class_cols:
        sent_class_col = col
        break

print(f"Sentiment class column: {sent_class_col if sent_class_col else 'None found'}")


Sentiment class column: classification


In [32]:
possible_date_cols = ["Date", "date", "timestamp", "time", "datetime"]
sent_time_col = next((col for col in possible_date_cols if col in sent.columns), None)
if sent_time_col is None:
    raise ValueError(f"No date column found in sentiment data. Found: {sent.columns.tolist()}")

possible_val_cols = ["Classification", "value", "sentiment", "score"]
sent_val_col = next((col for col in possible_val_cols if col in sent.columns), None)
if sent_val_col is None:
    raise ValueError(f"No sentiment value column found. Found: {sent.columns.tolist()}")

trade_time_col = next((col for col in possible_date_cols if col in trades.columns), None)
if trade_time_col is None:
    raise ValueError(f"No trade time column found in trades data. Found: {trades.columns.tolist()}")

# --- Convert datetime ---
sent[sent_time_col] = pd.to_datetime(sent[sent_time_col], errors='coerce', utc=True)
trades[trade_time_col] = pd.to_datetime(trades[trade_time_col], errors='coerce', utc=True)

# --- Sort sentiment ---
sent = sent.sort_values(by=sent_time_col).reset_index(drop=True)
sent_index = sent.set_index(sent_time_col)
sent_indexed = sent_index[[sent_val_col]].copy()

# --- Detect frequency ---
try:
    median_per_day = sent_indexed.resample("D").count().squeeze().median()
except Exception:
    median_per_day = 1
mapping_mode = "daily" if median_per_day <= 2 else "time_series"
print(f"Sentiment mapping mode: {mapping_mode} (median/day: {median_per_day})")

# --- Ensure UTC ---
if sent_indexed.index.tz is None:
    sent_indexed.index = sent_indexed.index.tz_localize("UTC")
else:
    sent_indexed.index = sent_indexed.index.tz_convert("UTC")

sent_indexed = sent_indexed.reset_index().rename(columns={
    sent_time_col: "_s_timestamp",
    sent_val_col: "_s_val"
})

# --- Convert sentiment classification to numeric if needed ---
if sent_indexed["_s_val"].dtype == object:
    mapping = {"Fear": 25, "Neutral": 50, "Greed": 75}  # Example mapping
    sent_indexed["_s_val_num"] = sent_indexed["_s_val"].map(mapping)
else:
    sent_indexed["_s_val_num"] = sent_indexed["_s_val"]

# --- Map trades to sentiment ---
if mapping_mode == "time_series":
    sent_times = sent_indexed["_s_timestamp"].values
    def map_prev_sent(trade_ts):
        if pd.isna(trade_ts):
            return pd.NaT
        idx = np.searchsorted(sent_times, trade_ts, side='right') - 1
        return sent_times[idx] if idx >= 0 else pd.NaT
    trades["_sent_time_mapped"] = trades[trade_time_col].apply(map_prev_sent)
    trades["_sent_time_mapped"] = pd.to_datetime(trades["_sent_time_mapped"], utc=True)
    trades = trades.merge(
        sent_indexed.rename(columns={"_s_timestamp": "_s_time"}),
        left_on="_sent_time_mapped", right_on="_s_time", how="left"
    )
else:
    trades["_trade_day"] = trades[trade_time_col].dt.floor("D")
    sent_indexed["_sent_day"] = sent_indexed["_s_timestamp"].dt.floor("D")
    sent_by_day = sent_indexed.groupby("_sent_day")["_s_val_num"].mean().reset_index()
    trades = trades.merge(sent_by_day, left_on="_trade_day", right_on="_sent_day", how="left")
    trades["_s_val_num"] = trades["_s_val_num"].fillna(trades["_s_val_num"].median())
    trades.drop(columns=["_sent_day"], inplace=True, errors="ignore")

mapped_count = trades["_s_val_num"].notna().sum()
print(f"Mapped sentiment to trades: {mapped_count} trades have sentiment value")

# --- Create sentiment bucket ---
trades["_s_bucket"] = pd.cut(
    trades["_s_val_num"],
    bins=[-1, 33, 66, 101],
    labels=["fear", "neutral", "greed"]
)

print(trades.head())

Sentiment mapping mode: daily (median/day: 1.0)
Mapped sentiment to trades: 0 trades have sentiment value
                                      account  coin  execution_price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   size_tokens  size_usd side     timestamp_ist  start_position direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   

#### 8) Market features: simple price returns & volatility (if price exists)

In [ ]:
if "_price" in trades.columns and trades["_price"].notna().sum() > 0:
    # Make sure trade_time_col exists
    possible_trade_time_cols = ["timestamp", "time", "trade_time", "datetime"]
    trade_time_col = next((col for col in possible_trade_time_cols if col in trades.columns), None)
    if trade_time_col is None:
        raise ValueError(f"No trade time column found in trades. Columns found: {trades.columns.tolist()}")

    # Ensure datetime format
    trades[trade_time_col] = pd.to_datetime(trades[trade_time_col])

    # Resample to hourly median prices
    price_hourly = (
        trades.set_index(trade_time_col)
        .resample("1H")["_price"]
        .median()
        .ffill()
        .to_frame(name="median_price")
    )
    price_hourly["log_return"] = np.log(price_hourly["median_price"]).diff()
    price_hourly["volatility_24h"] = price_hourly["log_return"].rolling(24).std() * (24 ** 0.5)
    price_hourly.reset_index(inplace=True)
    price_hourly.to_csv(os.path.join(OUTPUT_DIR, "price_hourly.csv"), index=False)

    # Convert both to same dtype (datetime64)
    price_times = price_hourly[trade_time_col].values.astype("datetime64[ns]")

    def map_price_feat(trade_ts):
        if pd.isna(trade_ts):
            return np.nan
        ts = np.datetime64(trade_ts)  # ensure same type
        idx = np.searchsorted(price_times, ts, side='right') - 1
        if idx >= 0:
            return price_hourly.iloc[idx]["median_price"]
        return np.nan

    trades["_mapped_median_price"] = trades[trade_time_col].apply(map_price_feat)

else:
    price_hourly = None


In [ ]:
if "_price" in trades.columns and trades["_price"].notna().sum() > 0:
    price_hourly = trades.set_index(trade_time_col).resample("1H")["_price"].median().ffill()
    price_hourly = price_hourly.to_frame(name="median_price")
    price_hourly["log_return"] = np.log(price_hourly["median_price"]).diff()
    price_hourly["volatility_24h"] = price_hourly["log_return"].rolling(24).std() * (24 ** 0.5)
    price_hourly.reset_index(inplace=True)
    price_hourly.to_csv(os.path.join(OUTPUT_DIR,"price_hourly.csv"), index=False)
    price_times = price_hourly[trade_time_col].values
    def map_price_feat(trade_ts):
        if pd.isna(trade_ts):
            return np.nan
        idx = np.searchsorted(price_times, trade_ts, side='right') - 1
        if idx >= 0:
            return price_hourly.iloc[idx]["median_price"]
        return np.nan
    trades["_mapped_median_price"] = trades[trade_time_col].apply(map_price_feat)
else:
    price_hourly = None


#### 9) Trader-level / rolling window features (if account exists)

In [41]:
if "_price" in trades.columns and trades["_price"].notna().sum() > 0:
    # Make sure trade_time_col exists
    possible_trade_time_cols = ["timestamp", "time", "trade_time", "datetime"]
    trade_time_col = next((col for col in possible_trade_time_cols if col in trades.columns), None)
    if trade_time_col is None:
        raise ValueError(f"No trade time column found in trades. Columns found: {trades.columns.tolist()}")

    # Ensure datetime format
    trades[trade_time_col] = pd.to_datetime(trades[trade_time_col])

    # Resample to hourly median prices
    price_hourly = (
        trades.set_index(trade_time_col)
        .resample("1H")["_price"]
        .median()
        .ffill()
        .to_frame(name="median_price")
    )
    price_hourly["log_return"] = np.log(price_hourly["median_price"]).diff()
    price_hourly["volatility_24h"] = price_hourly["log_return"].rolling(24).std() * (24 ** 0.5)
    price_hourly.reset_index(inplace=True)
    price_hourly.to_csv(os.path.join(OUTPUT_DIR, "price_hourly.csv"), index=False)

    # Convert both to same dtype (datetime64)
    price_times = price_hourly[trade_time_col].values.astype("datetime64[ns]")

    def map_price_feat(trade_ts):
        if pd.isna(trade_ts):
            return np.nan
        ts = np.datetime64(trade_ts)  # ensure same type
        idx = np.searchsorted(price_times, ts, side='right') - 1
        if idx >= 0:
            return price_hourly.iloc[idx]["median_price"]
        return np.nan

    trades["_mapped_median_price"] = trades[trade_time_col].apply(map_price_feat)

else:
    price_hourly = None


C:\Users\saram\AppData\Local\Temp\ipykernel_10372\1183215291.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample("1H")["_price"]
C:\Users\saram\AppData\Local\Temp\ipykernel_10372\1183215291.py:30: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts = np.datetime64(trade_ts)  # ensure same type


In [46]:
# Ensure columns exist
if account_col in trades.columns and trade_time_col in trades.columns:
    # Make sure time column is datetime
    trades[trade_time_col] = pd.to_datetime(trades[trade_time_col], errors="coerce")

    # Ensure account column consistent type
    trades[account_col] = trades[account_col].astype(str)

    # Sort by account and time
    trades = trades.sort_values([account_col, trade_time_col])

    def compute_rolling_features(df):
        # Must have datetime index for time-based rolling
        df = df.set_index(trade_time_col).sort_index()

        df["_winrate_7d"] = df["_profitable"].rolling("7d").mean()
        df["_avg_pnl_7d"] = df["_pnl"].rolling("7d").mean()
        df["_pnl_std_7d"] = df["_pnl"].rolling("7d").std()

        # Avoid divide-by-zero
        df["_sharpe_like_7d"] = df["_avg_pnl_7d"] / df["_pnl_std_7d"].replace(0, np.nan)

        df["_trades_7d"] = df["_pnl"].rolling("7d").count()

        return df.reset_index()

    # Apply per account
    trades = (
        trades.groupby(account_col, group_keys=False)
        .apply(compute_rolling_features)
        .reset_index(drop=True)
    )

else:
    print("No account column or trade time column found; skipping trader-level rolling features")


C:\Users\saram\AppData\Local\Temp\ipykernel_10372\3138670492.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_rolling_features)


#### 10) EDA: summary stats and plots

In [57]:
# Summary stats
stats = trades[["_pnl","_size","_price","_notional","_return_pct","_leverage","_holding_time_sec"]].describe().transpose()
stats.to_csv(os.path.join(OUTPUT_DIR,"trades_summary_stats.csv"))

In [65]:
# Plot histograms
def save_hist(series, fname, bins=100):
    v = series.dropna()
    if v.shape[0] == 0:
        return
    plt.figure(figsize=(8,4))
    plt.hist(v, bins=bins)
    plt.title(f"Distribution: {fname}")
    plt.xlabel("value")
    plt.ylabel("count")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,fname))
    plt.close()

save_hist(trades["_pnl"], "hist_pnl.png", bins=120)
save_hist(trades["_size"], "hist_size.png", bins=80)
save_hist(trades["_return_pct"], "hist_return_pct.png", bins=80)
save_hist(trades["_leverage"], "hist_leverage.png", bins=80)

In [61]:
import matplotlib.pyplot as plt
import os

# Plot histograms
def save_hist(series, fname, bins=100):
    # Drop NaNs and infinities
    v = series.replace([np.inf, -np.inf], np.nan).dropna()

    if v.shape[0] == 0:
        print(f"Skipping histogram for {fname} — no valid values.")
        return

    plt.figure(figsize=(8, 4))
    plt.hist(v, bins=bins, color="steelblue", edgecolor="black", alpha=0.7)
    plt.title(f"Distribution: {fname}")
    plt.xlabel("Value")
    plt.ylabel("Count")
    plt.tight_layout()

    # Ensure output directory exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    plt.savefig(os.path.join(OUTPUT_DIR, fname))
    plt.close()

# Call histograms for engineered features
save_hist(trades["_pnl"], "hist_pnl.png", bins=120)
save_hist(trades["_size"], "hist_size.png", bins=80)
save_hist(trades["_return_pct"], "hist_return_pct.png", bins=80)
save_hist(trades["_leverage"], "hist_leverage.png", bins=80)


Skipping histogram for hist_leverage.png — no valid values.


In [52]:
# Time-series daily aggregated PnL vs sentiment
trades["_day"] = trades[trade_time_col].dt.tz_convert("UTC").dt.floor("D")
sent["_day"] = sent[sent_time_col].dt.tz_convert("UTC").dt.floor("D")
daily_pnl = trades.groupby("_day")["_pnl"].sum().rename("daily_pnl").reset_index()
daily_sent = sent.groupby("_day")[sent_val_col].mean().rename("daily_sent").reset_index()
daily = pd.merge(daily_pnl, daily_sent, left_on="_day", right_on="_day", how="left")
daily.to_csv(os.path.join(OUTPUT_DIR,"daily_pnl_sent.csv"), index=False)


In [67]:
plt.figure(figsize=(10,4))
plt.plot(daily["_day"], daily["daily_pnl"])
plt.title("Daily aggregated PnL")
plt.xlabel("date")
plt.ylabel("daily_pnl")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,"daily_pnl.png"))
plt.close()




In [ ]:

plt.figure(figsize=(10,4))
plt.plot(daily["_day"], daily["daily_sent"])
plt.title("Daily sentiment (mean)")
plt.xlabel("date")
plt.ylabel("daily_sent")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,"daily_sent.png"))
plt.close()


In [ ]:
# Grouped analyses: PnL & win-rate by sentiment bucket
bucket_stats = trades.groupby("_s_bucket").agg(
    n_trades=("_pnl","count"),
    total_pnl=("_pnl","sum"),
    mean_pnl=("_pnl","mean"),
    median_pnl=("_pnl","median"),
    winrate=("_profitable","mean")
).reset_index()
bucket_stats.to_csv(os.path.join(OUTPUT_DIR,"bucket_stats.csv"), index=False)



In [ ]:
# Cross-section: mean return vs sentiment by account (if account exists)
if account_col:
    acc_stats = trades.groupby(account_col).agg(
        mean_return=("_return_pct","mean"),
        mean_sent=("_s_val","mean"),
        trades_count=("_pnl","count")
    ).reset_index()
    acc_stats.to_csv(os.path.join(OUTPUT_DIR,"account_sent_return.csv"), index=False)
    plt.figure(figsize=(8,6))
    plt.scatter(acc_stats["mean_sent"].fillna(0), acc_stats["mean_return"].fillna(0), s=10)
    plt.title("Account mean return vs mean sentiment")
    plt.xlabel("mean_sent")
    plt.ylabel("mean_return")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,"account_sent_return_scatter.png"))
    plt.close()

In [ ]:
# Correlation matrix for numeric features
num_cols = [c for c in ["_pnl","_size","_price","_notional","_return_pct","_leverage","_s_val"] if c in trades.columns]
corr = trades[num_cols].corr()
corr.to_csv(os.path.join(OUTPUT_DIR,"correlation_matrix.csv"))


In [ ]:
# Statistical test: Mann-Whitney U between fear and greed
if "_s_bucket" in trades.columns:
    fear = trades.loc[trades["_s_bucket"]=="fear", "_pnl"].dropna()
    greed = trades.loc[trades["_s_bucket"]=="greed", "_pnl"].dropna()
    if len(fear)>0 and len(greed)>0:
        try:
            mw_stat, mw_p = stats.mannwhitneyu(fear, greed, alternative='two-sided')
            with open(os.path.join(OUTPUT_DIR,"stat_test_mannwhitney.txt"), "w") as f:
                f.write(f"MW U stat={mw_stat}, p={mw_p}\n")
        except Exception as e:
            with open(os.path.join(OUTPUT_DIR,"stat_test_mannwhitney.txt"), "w") as f:
                f.write("Mann-Whitney failed: " + str(e) + "\n")


#### 11) Event study around extreme sentiment shifts (top/bottom 5%)

In [69]:
try:
    high_cut = sent[sent_val_col].quantile(0.95)
    low_cut  = sent[sent_val_col].quantile(0.05)
    high_events = sent.loc[sent[sent_val_col] >= high_cut, sent_time_col].sort_values().tolist()
    low_events  = sent.loc[sent[sent_val_col] <= low_cut, sent_time_col].sort_values().tolist()
    # limit number of events to analyze for speed
    high_events = high_events[:10]
    low_events = low_events[:10]
    def summarize_event_window(event_times, w_before=pd.Timedelta(days=3), w_after=pd.Timedelta(days=3)):
        summaries = []
        for et in event_times:
            start = et - w_before
            end = et + w_after
            subset = trades[(trades[trade_time_col] >= start) & (trades[trade_time_col] <= end)]
            summaries.append({
                "event_time": et,
                "n_trades": len(subset),
                "total_pnl": float(subset["_pnl"].sum()) if len(subset)>0 else 0.0,
                "mean_pnl": float(subset["_pnl"].mean()) if len(subset)>0 else 0.0
            })
        return pd.DataFrame(summaries)
    he_df = summarize_event_window(high_events)
    le_df = summarize_event_window(low_events)
    he_df.to_csv(os.path.join(OUTPUT_DIR,"event_high_summary.csv"), index=False)
    le_df.to_csv(os.path.join(OUTPUT_DIR,"event_low_summary.csv"), index=False)
except Exception as e:
    print("Event study failed:", e)


#### 12) Predictive modeling: predict whether a trade is profitable

In [ ]:
# Prefer _s_val, fallback to _s_val_num if needed
sentiment_col = "_s_val" if "_s_val" in trades.columns else "_s_val_num" if "_s_val_num" in trades.columns else None

# Base feature list
features = ["_size", "_price", "_notional", "_return_pct", sentiment_col, "_leverage"]

# Encode side as numeric if available
if "_side" in trades.columns:
    trades["_side_enc"] = trades["_side"].map(
        lambda x: 1 if isinstance(x, str) and ("buy" in x.lower() or "long" in x.lower())
        else (0 if isinstance(x, str) and ("sell" in x.lower() or "short" in x.lower())
        else np.nan)
    )
    features.append("_side_enc")

# Drop None and ensure all features exist in trades
features = [f for f in features if f is not None and f in trades.columns]

# Keep only numeric features for modeling
numeric_features = [f for f in features if pd.api.types.is_numeric_dtype(trades[f])]

# Prepare dataset
model_df = trades[numeric_features + ["_profitable", "_pnl"]].copy()
model_df = model_df.rename(columns={"_profitable": "profitable", "_pnl": "pnl"})
model_df = model_df.dropna(subset=["profitable"])

# Require both classes present
if len(model_df["profitable"].unique()) > 1 and len(model_df) > 100:
    X = model_df[numeric_features].to_numpy()
    y = model_df["profitable"].astype(int).to_numpy()

    # Impute missing values
    imputer = SimpleImputer(strategy="median")
    X_imp = imputer.fit_transform(X)

    # Scale
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imp)

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.25, random_state=42, stratify=y
    )

    # Random Forest
    rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    print("RF accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=0))

    # Permutation importance
    perm = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)
    perm_df = pd.DataFrame({
        "feature": numeric_features,
        "importance_mean": perm.importances_mean,
        "importance_std": perm.importances_std
    }).sort_values("importance_mean", ascending=False)
    perm_df.to_csv(os.path.join(OUTPUT_DIR, "permutation_importance.csv"), index=False)

    # Logistic Regression for interpretability
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    print("LR accuracy:", accuracy_score(y_test, y_pred_lr))
    print(classification_report(y_test, y_pred_lr, zero_division=0))

    # Save models and transformers
    joblib.dump(rf, os.path.join(OUTPUT_DIR, "rf_profitable.joblib"))
    joblib.dump(lr, os.path.join(OUTPUT_DIR, "lr_profitable.joblib"))
    joblib.dump(scaler, os.path.join(OUTPUT_DIR, "scaler.joblib"))
    joblib.dump(imputer, os.path.join(OUTPUT_DIR, "imputer.joblib"))

else:
    print("Not enough data/classes to train model (need both classes and >100 rows).")


#### 13) Auto-insights (simple heuristics)

In [104]:
insights = []

# Leverage vs PnL correlation
if "_leverage" in trades.columns and trades["_leverage"].notna().sum() > 10:
    corr = trades[["_leverage", "_pnl"]].dropna().corr().iloc[0, 1]
    insights.append(f"Correlation between leverage and PnL: {corr:.3f}")

# Win-rate by sentiment bucket
if "_s_bucket" in trades.columns:
    wr = trades.groupby("_s_bucket")["_profitable"].mean().to_dict()
    insights.append("Win-rate by sentiment bucket: " + ", ".join([f"{k}:{v:.2%}" for k, v in wr.items()]))

# Sentiment mapping count — check for either _s_val or _s_val_num
sentiment_col = "_s_val" if "_s_val" in trades.columns else "_s_val_num" if "_s_val_num" in trades.columns else None
if sentiment_col:
    mapped_count = trades[sentiment_col].notna().sum()
    insights.append(f"Mapped trades with sentiment: {int(mapped_count)} / {len(trades)}")
else:
    insights.append("No sentiment values found in dataset.")

# Save insights
pd.DataFrame({"insight": insights}).to_csv(os.path.join(OUTPUT_DIR, "auto_insights.csv"), index=False)


C:\Users\saram\AppData\Local\Temp\ipykernel_10372\4287449336.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  wr = trades.groupby("_s_bucket")["_profitable"].mean().to_dict()


#### 14) Save outputs

In [108]:
if "_s_bucket" in trades.columns and "_pnl" in trades.columns:
    bucket_stats = (
        trades.groupby("_s_bucket")["_pnl"]
        .agg(["count", "mean", "median", "std", "sum"])
        .reset_index()
        .sort_values("mean", ascending=False)
    )
    bucket_stats.to_csv(os.path.join(OUTPUT_DIR, "pnl_by_sentiment_bucket.csv"), index=False)


C:\Users\saram\AppData\Local\Temp\ipykernel_10372\2079304669.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trades.groupby("_s_bucket")["_pnl"]


In [110]:
# Save enriched trades (sample) and aggregates
trades.sample(n=min(5000, len(trades))).to_csv(os.path.join(OUTPUT_DIR,"trades_enriched_sample.csv"), index=False)
bucket_stats.to_csv(os.path.join(OUTPUT_DIR,"pnl_by_sentiment_bucket.csv"), index=False)
if 'acc_stats' in locals():
    acc_stats.to_csv(os.path.join(OUTPUT_DIR,"account_sent_return.csv"), index=False)

print("Completed. Outputs are in:", OUTPUT_DIR)
print("Key outputs saved: trades_enriched_sample.csv, daily_pnl_sent.csv, bucket_stats.csv, permutation_importance.csv (if model trained), models (if trained).")


Completed. Outputs are in: assignment folder
Key outputs saved: trades_enriched_sample.csv, daily_pnl_sent.csv, bucket_stats.csv, permutation_importance.csv (if model trained), models (if trained).
